In [42]:
#обычный Штакельберг для ромбовидной модели, 2 супервайзера - используем равновесие Нэша
#1 агент, 1 принципал
import numpy as np

#объявление переменных
N_s = 2 #количество супервайзеров

N_ag = 2  #по агенту - v         
N_s1, N_s2 = 2, 2    #по супервайзерам - u
N_pr1, N_pr2 = 2, 2 #по принципалу w

v_min = 1 
v_max = 5

u1_min, u2_min = 1, 1 
u1_max, u2_max = 5, 5

w1_min, w2_min = 1, 1
w1_max, w2_max = 5, 5

h_v = round((v_max - v_min)/N_ag, 3)

h_w1 = round((w1_max - w1_min)/N_pr1, 3)
h_w2 = round((w2_max - w2_min)/N_pr2, 3)

h_u1 = round((u1_max - u1_min)/N_s1, 3)
h_u2 = round((u2_max - u2_min)/N_s2, 3)

z0 = 0.1
z_abs = 0.6 #то с чем сравниваем - наш показатель "эффективности" - идеальный 

#определяем функции  
def J_0 (w1, w2, u1, u2, v):
    return round(w1*u1 + w2*u2 + v*v*w1*w2, 5)

def J1_1 (w1, w2, u1, u2, v):
    return round(w1*w2*u1*v, 5)
def J1_2(w1, w2, u1, u2, v):
    return round(w2*u2*u2/v, 5)

def J2 (w1, w2, u1, u2, v):
    return round((u1*w1*v - u2*w2)*v, 5)

def z_current(w1, w2, u1, u2, v):
     return round(z0 + 1.7 * (v)/10, 5)

#имитация    

def Shtackelberg_rhomb():

    print('шаги')
    print (h_v, h_w1, h_w2, h_u1, h_u2)
    qt = 0 
    J1_Nash_dic = {} #будем искать равновесие Нэша для супервайзеров. вид: [qt-key][J0, J1_1, J1_2, J2, w1,w2,u1,u2,v]
        
    
    for i_w1 in range (N_pr1 + 1):
        w1_cur = round(w1_min + i_w1 * h_w1, 5)

        for i_w2 in range (N_pr2 + 1):
            w2_cur = round(w2_min + i_w2 * h_w2, 5)
            
            for j_u1 in range (N_s1 + 1):
                u1_cur = round(u1_min + j_u1 * h_u1, 5)
                
                for j_u2 in range (N_s2 + 1):
                    u2_cur = round(u2_min + j_u2 * h_u2, 5)
                    
                    J2_lst = []
                    for k_v in range (N_ag + 1):
                        v_cur = round(v_min + k_v * h_v, 5)
                        J2_lst.append(round(J2 (w1_cur, w2_cur, u1_cur, u2_cur, v_cur), 5))
                        qt += 1
                    #ищем оптимальное по агенту    
                    J2_max = max(J2_lst)
                    v_opt = round(v_min + J2_lst.index(max(J2_lst)) * h_v, 5)
                        
                        
                    #тут проверяем пару (u1, u2) на Нэша 
                    J1_1_cur = round(J1_1(w1_cur, w2_cur, u1_cur, u2_cur, v_cur), 5)
                    J1_2_cur = round(J1_2(w1_cur, w2_cur, u1_cur, u2_cur, v_cur), 5)
                    
                    J1_1_Nash_lst = []
                    J1_2_Nash_lst = []
                    
                    print('current J1_1', J1_1_cur, 'J1_2', J1_2_cur)
                    Nash = False
                    
                    for k1_n in range (N_s1 + 1):
                        u1_n = round(u1_min + k1_n * h_u1, 5)
                        J1_1_Nash_lst.append(round(J1_1(w1_cur, w2_cur, u1_n, u2_cur, v_cur)))
                    print('набор значений J1 по управлениям', w1_cur, w2_cur,'u1_n', u2_cur, v_cur, 'выглядит следующим образом:', J1_1_Nash_lst)
                    max_J1_1_Nash = max(J1_1_Nash_lst)
                    print('max J1 Nash',  max_J1_1_Nash)
                    
                    if J1_1_cur >= max_J1_1_Nash:
                        Nash = True
                    else:
                        print('следующий набор значений из-за J1_1')
                    
                    if Nash:
                        for k2_n in range (N_s2 + 1):
                            u2_n = round(u2_min + k2_n * h_u2, 5)
                            J1_2_Nash_lst.append(round(J1_2(w1_cur, w2_cur, u1_cur, u2_n, v_cur)))
                        print('набор значений J1_2 по управлениям', w1_cur, w2_cur, u1_cur, 'u2_n', v_cur, 'выглядит следующим образом:', J1_1_Nash_lst)
                        max_J1_2_Nash = max(J1_2_Nash_lst)
                        print('max J2 Nash',  max_J1_2_Nash)
                        if J1_2_cur >= max_J1_2_Nash:
                            Nash = True
                        else:
                            print('следующий набор значений из-за J1_2')
                        
                    print(Nash)   
                    #если условия выполнены - добавляем в наш словарь
                    if Nash:
                        if qt not in J1_Nash_dic.keys():
                            J1_Nash_dic[qt] = [J1_1_cur, J1_2_cur, J2_max, w1_cur, w2_cur, u1_cur, u2_cur, v_cur]
                        print("значение z при оптимальных управлениях", z_current(w1_cur, w2_cur, u1_cur, u2_cur, v_cur))
                        if z_current(w1_cur, w2_cur, u1_cur, u2_cur, v_cur) < z_abs:
                            print("fale_z_current") #если не выполняется уловие 4 мы должны поменять управление то есть просто переход не след.шаг цикла по Т
                    print('******************************************************************')
    print ('общее количество вариантов наборов управлений', qt) 
    for item in J1_Nash_dic.keys():
        print (item, ':', J1_Nash_dic[item])
        #словари не упорядочены по порядку(ключу), поэтому печать не зависит от порядкового номера
    print('количество наборов оптимальных управлений агентов:', len(J1_Nash_dic))   

    print('значения ф. агента', J2_lst)
    print(J2_max, 'при', v_opt)
    
    
    #J1_1_cur, J1_2_cur, w1_cur, w2_cur, u1_cur, u2_cur, v_cur]
        
        
    J0_dic = {} 
    for item in J1_Nash_dic.keys():
        if item not in J0_dic.keys():
            J0_dic[item] = J_0(J1_Nash_dic[item][3], J1_Nash_dic[item][4], J1_Nash_dic[item][5],J1_Nash_dic[item][6], J1_Nash_dic[item][7])


    print('значения целевой ф. принципала')  
    max_J0 = max(J0_dic.values())
    min_J0 = min(J0_dic.values())
    print('min', min_J0,'max', max_J0)


Shtackelberg_rhomb()


шаги
2.0 2.0 2.0 2.0 2.0
current J1_1 5.0 J1_2 0.2
набор значений J1 по управлениям 1.0 1.0 u1_n 1.0 5.0 выглядит следующим образом: [5, 15, 25]
max J1 Nash 25
следующий набор значений из-за J1_1
False
******************************************************************
current J1_1 5.0 J1_2 1.8
набор значений J1 по управлениям 1.0 1.0 u1_n 3.0 5.0 выглядит следующим образом: [5, 15, 25]
max J1 Nash 25
следующий набор значений из-за J1_1
False
******************************************************************
current J1_1 5.0 J1_2 5.0
набор значений J1 по управлениям 1.0 1.0 u1_n 5.0 5.0 выглядит следующим образом: [5, 15, 25]
max J1 Nash 25
следующий набор значений из-за J1_1
False
******************************************************************
current J1_1 15.0 J1_2 0.2
набор значений J1 по управлениям 1.0 1.0 u1_n 1.0 5.0 выглядит следующим образом: [5, 15, 25]
max J1 Nash 25
следующий набор значений из-за J1_1
False
*****************************************************************